## Testes com modelos para classificar o tipo de falha

No notebook ```modelo_classificacao1.ipynb```, fizemos testes para classificar o tipo de fala do S_GROUP_ID_1, utilizando diferentes redes neurais. Porém, aqui realizaremos testes com modelos de Regressão Logística, XGBoost e Random Forest, a fim de testar qual terá um melhor desempenho para esse conjunto de dados. 

In [18]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [19]:
# carregando o dataframe
df= pd.read_csv("data/df_torques_falhas.csv")
df

,KNR,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,...,COR_6KA1,COR_6UA1,COR_A1A1,COR_K2A1,COR_K2K2,COR_O7O7,MOTOR_CWL,MOTOR_CWS,MOTOR_DHS,MOTOR_DRP
0,2023-2016173,0.616167,0.0,0.0,0.264989,0.074849,0.884665,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-2026098,0.616167,0.0,0.0,0.462990,0.093920,0.919817,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-2026162,0.616167,0.0,0.0,0.330866,0.094345,0.921463,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-2026175,0.616167,0.0,0.0,0.436343,0.094345,0.894115,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-2026215,0.616167,0.0,0.0,0.300888,0.094345,0.911870,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49186,2024-2976009,0.616167,0.0,0.0,0.195855,0.430977,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
49187,2024-2976010,0.616167,0.0,0.0,0.177955,0.429747,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49188,2024-2976011,0.616167,0.0,0.0,0.184092,0.424991,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49189,2024-2976012,0.616167,0.0,0.0,0.192237,0.436295,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
df.columns

Index(['KNR', '%', 'CLICKS', 'DEG', 'GRAD', 'NM', 'V', '\U00B0', 'KG', 'MIN',
       'MM', 'ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK',
       'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA',
       'S_GROUP_ID_-2', 'S_GROUP_ID_1', 'S_GROUP_ID_2', 'S_GROUP_ID_4',
       'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137', 'S_GROUP_ID_140',
       'S_GROUP_ID_9830946', 'COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1',
       'COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1',
       'COR_K2A1', 'COR_K2K2', 'COR_O7O7', 'MOTOR_CWL', 'MOTOR_CWS',
       'MOTOR_DHS', 'MOTOR_DRP'],
      dtype='object')

In [21]:
# Converter as colunas para binário
cols = ['S_GROUP_ID_1', 'S_GROUP_ID_2', 'S_GROUP_ID_4', 'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137', 'S_GROUP_ID_140', 'S_GROUP_ID_9830946']

# Criar a condição para converter as colunas para binário
df[cols] = (df[cols] > 0).astype(int)

# Exibir as colunas convertidas
df[cols]



,S_GROUP_ID_1,S_GROUP_ID_2,S_GROUP_ID_4,S_GROUP_ID_5,S_GROUP_ID_133,S_GROUP_ID_137,S_GROUP_ID_140,S_GROUP_ID_9830946
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
49186,1,1,1,1,0,0,0,0
49187,0,1,1,1,0,0,1,0
49188,1,1,0,1,0,0,1,0
49189,1,1,1,1,1,0,0,0


In [22]:
# Excluindo as cplunas com dados NaN
df = df.dropna()
df.shape

(40152, 49)

In [23]:
df = df.drop(columns = ['COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1','COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1', 'COR_K2A1', 'COR_K2K2', 'COR_O7O7', 'MOTOR_CWL', 'MOTOR_CWS','MOTOR_DHS', 'MOTOR_DRP', 'S_GROUP_ID_-2'])

Modelo para teste

In [24]:
# Modelo random forest
model_tree = RandomForestClassifier(n_estimators=100, random_state=42)

# # Criando o modelo XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc')

# Modelo de regressão logística
logistic_model = LogisticRegression()


In [25]:
# Converter as colunas para binário
cols = ['S_GROUP_ID_1', 'S_GROUP_ID_2','S_GROUP_ID_4','S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137','S_GROUP_ID_140', 'S_GROUP_ID_9830946']


## Testes com XGBoost

In [26]:
# Iterando sobre cada grupo
for col in cols:
    print(f"Treinando o modelo para {col}")

    # Separando as features (X) e o target (y) para o grupo atual
    X = df.drop(columns=[col, 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
    y = df[col]

    # Separando as classes majoritária e minoritária com base no target y2
    X_majority = X[y == 0]
    X_minority = X[y == 1]

    # Definindo o número de clusters como o mínimo entre as duas classes
    n_clusters = min(len(X_majority), len(X_minority))

    # Aplicando o K-Means à classe majoritária
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    kmeans.fit(X_majority)

    # Pegando os centróides dos clusters
    X_majority_centroids = kmeans.cluster_centers_

    # Combinando os centróides com a classe minoritária
    X = np.vstack((X_majority_centroids, X_minority))
    y = np.hstack((np.zeros(len(X_majority_centroids)), np.ones(len(X_minority))))

    # Separando em dados de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

     
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Fazendo as previsões
    y_pred = model.predict(X_test)
    
    # Converter as probabilidades em classes binárias (0 ou 1) ajustando
    y_pred_classes= (y_pred > 0.5).astype(int)

    # Calcular as principais métricas
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    print(f"resultados da coluna{col}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Matriz de Confusão
    print(confusion_matrix(y_test, y_pred_classes))

    # Relatório de Classificação
    print(classification_report(y_test, y_pred_classes))

    nome_arquivo = f'XGBOOST:{col}.h5'  # Gera o nome do arquivo dinamicamente
    joblib.dump(model, nome_arquivo) # Salva o modelo com o nome gerado


Treinando o modelo para S_GROUP_ID_1


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:47:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


resultados da colunaS_GROUP_ID_1
Accuracy: 0.6074
Precision: 0.6418
Recall: 0.8326
F1-Score: 0.7249
[[ 724 2318]
 [ 835 4154]]
              precision    recall  f1-score   support

         0.0       0.46      0.24      0.31      3042
         1.0       0.64      0.83      0.72      4989

    accuracy                           0.61      8031
   macro avg       0.55      0.54      0.52      8031
weighted avg       0.57      0.61      0.57      8031

Treinando o modelo para S_GROUP_ID_2


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:47:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


resultados da colunaS_GROUP_ID_2
Accuracy: 0.9025
Precision: 0.9040
Recall: 0.9982
F1-Score: 0.9487
[[   1  770]
 [  13 7247]]
              precision    recall  f1-score   support

         0.0       0.07      0.00      0.00       771
         1.0       0.90      1.00      0.95      7260

    accuracy                           0.90      8031
   macro avg       0.49      0.50      0.48      8031
weighted avg       0.82      0.90      0.86      8031

Treinando o modelo para S_GROUP_ID_4


...

resultados da colunaS_GROUP_ID_137
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
[[13  0]
 [ 0 12]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        13
         1.0       1.00      1.00      1.00        12

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

Treinando o modelo para S_GROUP_ID_140


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:49:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


resultados da colunaS_GROUP_ID_140
Accuracy: 0.9727
Precision: 0.9574
Recall: 0.9890
F1-Score: 0.9730
[[88  4]
 [ 1 90]]
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97        92
         1.0       0.96      0.99      0.97        91

    accuracy                           0.97       183
   macro avg       0.97      0.97      0.97       183
weighted avg       0.97      0.97      0.97       183

Treinando o modelo para S_GROUP_ID_9830946


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:50:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


resultados da colunaS_GROUP_ID_9830946
Accuracy: 0.9578
Precision: 0.9564
Recall: 0.9592
F1-Score: 0.9578
[[329  15]
 [ 14 329]]
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96       344
         1.0       0.96      0.96      0.96       343

    accuracy                           0.96       687
   macro avg       0.96      0.96      0.96       687
weighted avg       0.96      0.96      0.96       687



## Testes com Random Forest

In [27]:
# Iterando sobre cada grupo
for col in cols:
    print(f"Treinando o modelo para {col}")

    # Separando as features (X) e o target (y) para o grupo atual
    X = df.drop(columns=[col, 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
    y = df[col]

    # Separando as classes majoritária e minoritária com base no target y2
    X_majority = X[y == 0]
    X_minority = X[y == 1]

    # Definindo o número de clusters como o mínimo entre as duas classes
    n_clusters = min(len(X_majority), len(X_minority))

    # Aplicando o K-Means à classe majoritária
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    kmeans.fit(X_majority)

    # Pegando os centróides dos clusters
    X_majority_centroids = kmeans.cluster_centers_

    # Combinando os centróides com a classe minoritária
    X = np.vstack((X_majority_centroids, X_minority))
    y = np.hstack((np.zeros(len(X_majority_centroids)), np.ones(len(X_minority))))

    # Separando em dados de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

     
    # Treinando o modelo
    model_tree.fit(X_train, y_train)
    
    # Fazendo as previsões
    y_pred = model_tree.predict(X_test)
    
    # Converter as probabilidades em classes binárias (0 ou 1) ajustando
    y_pred_classes= (y_pred > 0.5).astype(int)

    # Calcular as principais métricas
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    print(f"resultados da coluna{col}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Matriz de Confusão
    print(confusion_matrix(y_test, y_pred_classes))

    # Relatório de Classificação
    print(classification_report(y_test, y_pred_classes))

    nome_arquivo = f'Random Forest {col}.h5'  # Gera o nome do arquivo dinamicamente
    joblib.dump(model_tree, nome_arquivo) # Salva o modelo com o nome gerado


Treinando o modelo para S_GROUP_ID_1
resultados da colunaS_GROUP_ID_1
Accuracy: 0.6150
Precision: 0.6399
Recall: 0.8695
F1-Score: 0.7373
[[ 601 2441]
 [ 651 4338]]
              precision    recall  f1-score   support

         0.0       0.48      0.20      0.28      3042
         1.0       0.64      0.87      0.74      4989

    accuracy                           0.61      8031
   macro avg       0.56      0.53      0.51      8031
weighted avg       0.58      0.61      0.56      8031

Treinando o modelo para S_GROUP_ID_2
resultados da colunaS_GROUP_ID_2
Accuracy: 0.9037
Precision: 0.9041
Recall: 0.9996
F1-Score: 0.9494
[[   1  770]
 [   3 7257]]
              precision    recall  f1-score   support

         0.0       0.25      0.00      0.00       771
         1.0       0.90      1.00      0.95      7260

    accuracy                           0.90      8031
   macro avg       0.58      0.50      0.48      8031
weighted avg       0.84      0.90      0.86      8031

Treinando o modelo

## Testes com Regressão Logística

In [28]:
# Iterando sobre cada grupo
for col in cols:
    print(f"Treinando o modelo para {col}")

    # Separando as features (X) e o target (y) para o grupo atual
    X = df.drop(columns=[col, 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
    y = df[col]

    # Separando as classes majoritária e minoritária com base no target y2
    X_majority = X[y == 0]
    X_minority = X[y == 1]

    # Definindo o número de clusters como o mínimo entre as duas classes
    n_clusters = min(len(X_majority), len(X_minority))

    # Aplicando o K-Means à classe majoritária
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    kmeans.fit(X_majority)

    # Pegando os centróides dos clusters
    X_majority_centroids = kmeans.cluster_centers_

    # Combinando os centróides com a classe minoritária
    X = np.vstack((X_majority_centroids, X_minority))
    y = np.hstack((np.zeros(len(X_majority_centroids)), np.ones(len(X_minority))))

    # Separando em dados de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

     
    # Treinando o modelo
    logistic_model.fit(X_train, y_train)
    
    # Fazendo as previsões
    y_pred = logistic_model.predict(X_test)
    
    # Converter as probabilidades em classes binárias (0 ou 1) ajustando
    y_pred_classes= (y_pred > 0.5).astype(int)

    # Calcular as principais métricas
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    print(f"resultados da coluna{col}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Matriz de Confusão
    print(confusion_matrix(y_test, y_pred_classes))

    # Relatório de Classificação
    print(classification_report(y_test, y_pred_classes))

    nome_arquivo = f'Regressao Logistica {col}.h5'  # Gera o nome do arquivo dinamicamente
    joblib.dump(model, nome_arquivo) # Salva o modelo com o nome gerado


Treinando o modelo para S_GROUP_ID_1


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


resultados da colunaS_GROUP_ID_1
Accuracy: 0.6207
Precision: 0.6229
Recall: 0.9868
F1-Score: 0.7637
[[  62 2980]
 [  66 4923]]
              precision    recall  f1-score   support

         0.0       0.48      0.02      0.04      3042
         1.0       0.62      0.99      0.76      4989

    accuracy                           0.62      8031
   macro avg       0.55      0.50      0.40      8031
weighted avg       0.57      0.62      0.49      8031

Treinando o modelo para S_GROUP_ID_2


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is 

resultados da colunaS_GROUP_ID_2
Accuracy: 0.9040
Precision: 0.9040
Recall: 1.0000
F1-Score: 0.9496
[[   0  771]
 [   0 7260]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       771
         1.0       0.90      1.00      0.95      7260

    accuracy                           0.90      8031
   macro avg       0.45      0.50      0.47      8031
weighted avg       0.82      0.90      0.86      8031

Treinando o modelo para S_GROUP_ID_4


...

/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


resultados da colunaS_GROUP_ID_137
Accuracy: 0.4800
Precision: 0.4615
Recall: 0.5000
F1-Score: 0.4800
[[6 7]
 [6 6]]
              precision    recall  f1-score   support

         0.0       0.50      0.46      0.48        13
         1.0       0.46      0.50      0.48        12

    accuracy                           0.48        25
   macro avg       0.48      0.48      0.48        25
weighted avg       0.48      0.48      0.48        25

Treinando o modelo para S_GROUP_ID_140


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


resultados da colunaS_GROUP_ID_140
Accuracy: 0.6120
Precision: 0.6000
Recall: 0.6593
F1-Score: 0.6283
[[52 40]
 [31 60]]
              precision    recall  f1-score   support

         0.0       0.63      0.57      0.59        92
         1.0       0.60      0.66      0.63        91

    accuracy                           0.61       183
   macro avg       0.61      0.61      0.61       183
weighted avg       0.61      0.61      0.61       183

Treinando o modelo para S_GROUP_ID_9830946
resultados da colunaS_GROUP_ID_9830946
Accuracy: 0.6667
Precision: 0.6351
Recall: 0.7813
F1-Score: 0.7007
[[190 154]
 [ 75 268]]
              precision    recall  f1-score   support

         0.0       0.72      0.55      0.62       344
         1.0       0.64      0.78      0.70       343

    accuracy                           0.67       687
   macro avg       0.68      0.67      0.66       687
weighted avg       0.68      0.67      0.66       687



/home/barretob/Modelos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
